<a href="https://colab.research.google.com/github/AnwarOsmani/FaceRecognitionProject/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Data Preparation with more augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomCrop(32, padding=4),  # Random crop
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Model Definition with additional Conv Layers, Batch Normalization, and more Dropout
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)  # Additional convolution
        self.bn4 = nn.BatchNorm2d(256)  # Batch Normalization for the new layer
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(256 * 2 * 2, 512)  # Adjusted for new convolution layer
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))  # New convolution layer
        x = x.view(-1, 256 * 2 * 2)  # Adjusted for the new conv layer
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Model, Loss Function, and Optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop with more epochs
epochs = 40  # Increased number of epochs
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(trainloader)}")

# Evaluate on Test Data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# Save the Trained Model
torch.save(model.state_dict(), 'cnn_cifar10.pth')
print("Model saved as cnn_cifar10.pth")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/40, Loss: 1.5992137077824233
Epoch 2/40, Loss: 1.2483443392969458
Epoch 3/40, Loss: 1.0926013864062327
Epoch 4/40, Loss: 0.9890044439021889
Epoch 5/40, Loss: 0.9184940310237962
Epoch 6/40, Loss: 0.8622032237784637
Epoch 7/40, Loss: 0.824799417077428
Epoch 8/40, Loss: 0.7807503020214608
Epoch 9/40, Loss: 0.7589170508982276
Epoch 10/40, Loss: 0.7234487763374967
Epoch 11/40, Loss: 0.6964443902987654
Epoch 12/40, Loss: 0.6782793647531048
Epoch 13/40, Loss: 0.6565317178473753
Epoch 14/40, Loss: 0.6413453148149163
Epoch 15/40, Loss: 0.6277482290859417
Epoch 16/40, Loss: 0.6085263859966527
Epoch 17/40, Loss: 0.598341470026909
Epoch 18/40, Loss: 0.583225298377559
Epoch 19/40, Loss: 0.5731234988745522
Epoch 20/40, Loss: 0.564718511384314
Epoch 21/40, Loss: 0.5494528618424445
Epoch 22/40, Loss: 0.539250733869155
Epoch 23/40, Loss: 0.5354446681487895
Epoch 24/40, Loss: 0.5240206128305487
Epoch 25/40, Loss: 0.51420